# TF OpenPose Notebook

This is a jupyter-notebook implementation of the [tf-pose-estimation/run_video.py](https://github.com/ildoonet/tf-pose-estimation/blob/master/run_video.py) script.

### Acknowledgement

Many thanks to https://github.com/ildoonet/tf-pose-estimation for providing pretrained model as well as the image preprocessing/pose estimating infrastructure.

In [1]:
import argparse
import logging
import time

import cv2
import numpy as np

from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




In [2]:
logger = logging.getLogger('TfPoseEstimator-Video')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)



In [3]:
def main(argString = '--video ./images/video.avi --resolution 656x368 --model inf'):
    
    fps_time = 0
    
    parser = argparse.ArgumentParser(description='tf-pose-estimation Video')
    parser.add_argument('--video', type=str, default='')
    parser.add_argument('--resolution', type=str, default='432x368', help='network input resolution. default=432x368')
    parser.add_argument('--model', type=str, default='mobilenet_thin', help='cmu / mobilenet_thin / mobilenet_v2_large / mobilenet_v2_small')
    parser.add_argument('--show-process', type=bool, default=False,
                        help='for debug purpose, if enabled, speed for inference is dropped.')
    parser.add_argument('--showBG', type=bool, default=True, help='False to show skeleton only.')
    args = parser.parse_args(argString.split())

    logger.debug('initialization %s : %s' % (args.model, get_graph_path(args.model)))
    w, h = model_wh(args.resolution)
    e = TfPoseEstimator(get_graph_path(args.model), target_size=(w, h))
    cap = cv2.VideoCapture(args.video)
    out = cv2.VideoWriter('./images/output.avi', cv2.VideoWriter_fourcc(*'XVID'), 10.0, (int(cap.get(3)), int(cap.get(4))))

    if cap.isOpened() is False:
        print("Error opening video stream or file")
    while cap.isOpened():
        ret_val, image = cap.read()
        
        try:
            humans = e.inference(image, upsample_size=4.0)
        except:
            logger.debug('BREAK on Exception: The image is not valid. Please check your image exists.')
            break
        if not args.showBG:
            image = np.zeros(image.shape)
        image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

        cv2.putText(image, "FPS: %f" % (1.0 / (time.time() - fps_time)), (10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        logger.debug("FPS: %f" % (1.0 / (time.time() - fps_time)))
        #cv2.imshow('tf-pose-estimation result', image)
        out.write(image)
        fps_time = time.time()
        #if cv2.waitKey(1) == 27:
        #    break
    cap.release()
    out.release()
    cv2.destroyAllWindows()

   


In [4]:
main('--video ./images/video.avi --resolution 656x368 --model inf')

[2020-07-21 14:07:18,445] [TfPoseEstimator-Video] [DEBUG] initialization inf : /home/ubuntu/openpose/tf-pose-estimation/models/graph/inf/graph_opt.pb
[2020-07-21 14:07:18,445] [TfPoseEstimator-Video] [DEBUG] initialization inf : /home/ubuntu/openpose/tf-pose-estimation/models/graph/inf/graph_opt.pb
2020-07-21 14:07:18,445 DEBUG initialization inf : /home/ubuntu/openpose/tf-pose-estimation/models/graph/inf/graph_opt.pb
[2020-07-21 14:07:18,447] [TfPoseEstimator] [INFO] loading graph from /home/ubuntu/openpose/tf-pose-estimation/models/graph/inf/graph_opt.pb(default size=656x368)
2020-07-21 14:07:18,447 INFO loading graph from /home/ubuntu/openpose/tf-pose-estimation/models/graph/inf/graph_opt.pb(default size=656x368)


2020-07-21 14:07:18,448 WARNING From /home/ubuntu/openpose/tf-pose-estimation/tf_pose/estimator.py:311: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



2020-07-21 14:07:18,449 WARNING From /home/ubuntu/openpose/tf-pose-estimation/tf_pose/estimator.py:312: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef instead.



2020-07-21 14:07:18,677 WARNING From /home/ubuntu/openpose/tf-pose-estimation/tf_pose/estimator.py:330: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



2020-07-21 14:07:19,047 WARNING From /home/ubuntu/openpose/tf-pose-estimation/tf_pose/estimator.py:332: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



...

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2020-07-21 14:07:19,318 WARNING From /home/ubuntu/openpose/tf-pose-estimation/tf_pose/estimator.py:354: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2020-07-21 14:07:19,319 WARNING From /home/ubuntu/openpose/tf-pose-estimation/tf_pose/estimator.py:359: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.



2020-07-21 14:07:19,320 WARNING From /home/ubuntu/openpose/tf-pose-estimation/tf_pose/estimator.py:360: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



2020-07-21 14:07:19,321 WARNING From /home/ubuntu/openpose/tf-pose-estimation/tf_pose/estimator.py:362: The name tf.report_uninitialized_variables is deprecated. Please use tf.compat.v1.report_uninitialized_variables instead.

[2020-07-21 14:07:23,019] [TfPoseEstimator-Video] [DEBUG] FPS: 0.000000
[2020-07-21 14:07:23,019] [TfPoseEstimator-Video] [DEBUG] FPS: 0.000000
2020-07-21 14:07:23,019 DEBUG FPS: 0.000000
[2020-07-21 14:07:23,119] [TfPoseEstimator-Video] [DEBUG] FPS: 10.889544
[2020-07-21 14:07:23,119] [TfPoseEstimator-Video] [DEBUG] FPS: 10.889544
2020-07-21 14:07:23,119 DEBUG FPS: 10.889544
[2020-07-21 14:07:23,225] [TfPoseEstimator-Video] [DEBUG] FPS: 10.243576
[2020-07-21 14:07:23,225] [TfPoseEstimator-Video] [DEBUG] FPS: 10.243576
2020-07-21 14:07:23,225 DEBUG FPS: 10.243576
[2020-07-21 14:07:23,328] [TfPoseEstimator-Video] [DEBUG] FPS: 10.532815
[2020-07-21 14:07:23,328] [TfPoseEstimator-Video] [DEBUG] FPS: 10.532815
2020-07-21 14:07:23,328 DEBUG FPS: 10.532815
[2020-07-21 

In [5]:
from IPython.display import Video

Video('./images/output.avi',filename='./images/output.avi')